In [ ]:
# necessary libraries
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError
import pandas as pd
import json
import hashlib

In [ ]:
# client configuration
client = MailchimpMarketing.Client()
client.set_config({
    "api_key": "API_KEY",
    "server": "us10"
})

# get current audience id
list_id = "<audience-id>" #all FREE reports audience


In [ ]:
response = client.lists.get_list(list_id)
# response

In [ ]:
# get current audience data
all_members = []

try:
    offset = 0
    limit = 1000
    while True:
        response = client.lists.get_list_members_info(list_id, count=limit, offset=offset)
        members = response["members"]
        
        # Check for members
        if members:
            all_members.extend(members)
            offset += limit
        else:
            break

    print("Total members:", len(all_members))
    # print(all_members)

except MailchimpMarketing.ApiClientError as error:
    print("Error: {}".format(error.text))

members_data = []

for member in all_members:
    if member["list_id"] == list_id:
        member_info = {
            "id": member['id'],
            "email_address": member['email_address'],
            "full_name": member['full_name'],
                "FNAME": member['merge_fields']['FNAME'],
                "LNAME": member['merge_fields']['LNAME'],
                "ADDRESS": member['merge_fields']['ADDRESS'],
                "PHONE": member['merge_fields']['PHONE'],
                "FIRMNAME": member['merge_fields']['FIRMNAME'],
                "FIRMCLASS": member['merge_fields']['FIRMCLASS'],
            "timestamp_signup": member['timestamp_signup'],
            "timestamp_opt": member['timestamp_opt'],
            "last_changed": member['last_changed'],
            "status": member['status'],
            "avg_open_rate": member['stats']['avg_open_rate'],
            "avg_click_rate": member['stats']['avg_click_rate']
        }
        members_data.append(member_info)
import json
 # Print the formatted JSON
# print(json.dumps(members_data, indent=4))


In [ ]:
emails = [member['email_address'] for member in members_data]
email_ids = [member['id'] for member in members_data]

In [ ]:
df = pd.DataFrame(members_data)
object_columns = ['email_address', 'full_name', 'FNAME', 'LNAME', 'ADDRESS', 'PHONE', 'FIRMNAME', 'FIRMCLASS', 'timestamp_signup', 'timestamp_opt', 'last_changed', 'status']
df[object_columns] = df[object_columns].astype(str)

In [ ]:
# save member data
df = spark.createDataFrame(df)
table_path = "common.MailChimp_AFDR_Members"
df.write.format("delta").mode("append").saveAsTable(table_path)

# df.createOrReplaceTempView("temp_table")

In [ ]:
# get campaign data
from mailchimp_marketing.api_client import ApiClientError
import json  # import json if you're using json.dumps

all_campaigns = []  
try:
    offset = 0
    limit = 1000
    while True:
        response = client.campaigns.list(count=limit, offset=offset)
        campaigns = response["campaigns"]
        
        # Check for campaigns
        if campaigns:
            all_campaigns.extend(campaigns)
            offset += limit
        else:
            break
except ApiClientError as error:  
    print("Error: {}".format(error.text))

campaign_data = []
for campaign in all_campaigns:
    if campaign["recipients"]["list_id"] == list_id:
        campaign_info = {
            "id": campaign['id'],
            "title": campaign["settings"]["title"],
            "list_id": campaign["recipients"]["list_id"],
            "list_name": campaign["recipients"]["list_name"],
            "create_time": campaign['create_time'],
            "send_time": campaign['send_time'],
        }

        # Try to update 'subject_line'
        try:
            campaign_info["subject_line"] = campaign["settings"]["subject_line"]
        except KeyError:
            campaign_info["subject_line"] = None

        # Try to update report_summary fields
        try:
            campaign_info.update({
                "opens": campaign['report_summary']['opens'],
                "unique_opens": campaign['report_summary']['unique_opens'],
                "open_rate": campaign['report_summary']['open_rate'],
                "clicks": campaign['report_summary']['clicks'],
                "subscriber_clicks": campaign['report_summary']['subscriber_clicks'],
                "click_rate": campaign['report_summary']['click_rate']
            })
        except Exception as e:
            print(f"Exception while processing campaign {campaign['id']}: {e}")

        campaign_data.append(campaign_info)

# print(json.dumps(campaign_data, indent=4))

In [ ]:
df = pd.DataFrame(campaign_data)
df = spark.createDataFrame(df)
df.createOrReplaceTempView("temp_table2")

In [ ]:
%sql
-- merge campaign data into current campaign data
MERGE INTO common.MailChimp_AFDR_Campaign_SummaryData AS target
USING temp_table2 AS source
ON target.id = source.id
WHEN MATCHED THEN
  UPDATE SET 
    target.title = source.title,
    target.list_id = source.list_id,
    target.list_name = source.list_name,
    target.subject_line = source.subject_line,
    target.create_time = source.create_time,
    target.send_time = source.send_time,
    target.opens = source.opens,
    target.unique_opens = source.unique_opens,
    target.open_rate = source.open_rate,
    target.clicks = source.clicks,
    target.subscriber_clicks = source.subscriber_clicks,
    target.click_rate = source.click_rate
WHEN NOT MATCHED THEN
  INSERT (id, title, list_id, list_name, subject_line, create_time, send_time, opens, unique_opens, open_rate, clicks, subscriber_clicks, click_rate) 
  VALUES (
    source.id,
    source.title,
    source.list_id,
    source.list_name,
    source.subject_line,
    source.create_time,
    source.send_time,
    source.opens,
    source.unique_opens,
    source.open_rate,
    source.clicks,
    source.subscriber_clicks,
    source.click_rate
  );


In [ ]:
%sql
OPTIMIZE common.MailChimp_AFDR_Campaign_SummaryData ZORDER BY (send_time, clicks, opens);

In [ ]:
all_reports = []  
try:
    offset = 0
    limit = 1000
    while True:
        response = client.reports.get_all_campaign_reports(count=limit, offset=offset)
        reports = response['reports']
        # Check for campaigns
        if reports:
            all_reports.extend(reports)
            offset += limit
        else:
            break
    # print(json.dumps(all_reports, indent=2))

except MailchimpMarketing.ApiClientError as error:
    print("Error: {}".format(error.text))

report_data = []
for report in all_reports:
    if report['list_id']==list_id:
        report_info = {
        "id": report['id'],
        "campaign_title": report['campaign_title'],
        "type": report["type"],
        "list_id": report['list_id'],
        "list_is_active": report['list_is_active'],
        "list_name": report['list_name'],
        "subject_line": report['subject_line'],
        "preview_text": report['preview_text'],
        "emails_sent": report['emails_sent'],
        "abuse_reports": report['abuse_reports'],
        "unsubscribed": report['unsubscribed'],
        "send_time": report['send_time'],
        "opens_total": report["opens"]['opens_total'],
        "unique_opens": report["opens"]['unique_opens'],
        "open_rate": report["opens"]['open_rate'],
        "last_open": report["opens"]['last_open'],
        "clicks_total": report["clicks"]['clicks_total'],
        "unique_clicks": report["clicks"]['unique_clicks'],
        "unique_subscriber_clicks": report["clicks"]['unique_subscriber_clicks'],
        "click_rate": report["clicks"]['click_rate'],
        "last_click": report["clicks"]['last_click'],
            "type": report['industry_stats']['type'],
            "open_rate": report['industry_stats']['open_rate'],
            "click_rate": report['industry_stats']['click_rate'],
            "bounce_rate": report['industry_stats']['bounce_rate'],
            "unopen_rate": report['industry_stats']['unopen_rate'],
            "unsub_rate": report['industry_stats']['unsub_rate'],
            "abuse_rate": report['industry_stats']['abuse_rate'],
            "sub_rate": report['list_stats']['sub_rate'],
            "unsub_rate": report['list_stats']['unsub_rate'],
            "open_rate": report['list_stats']['open_rate'],
            "click_rate": report['list_stats']['click_rate']
        }
        report_data.append(report_info)

# print(json.dumps(report_data, indent=4))       

In [ ]:
cids = [cd['id'] for cd in campaign_data]     

In [ ]:
df = pd.DataFrame(report_data)
df = spark.createDataFrame(df)
df.createOrReplaceTempView("temp_table3")

In [ ]:
%sql
MERGE INTO common.MailChimp_AFDR_Campaign_ReportData AS target
USING temp_table3 AS source
ON target.id = source.id
WHEN MATCHED THEN
  UPDATE SET 
    target.campaign_title = source.campaign_title,
    target.type = source.type,
    target.list_id = source.list_id,
    target.list_is_active = source.list_is_active,
    target.list_name = source.list_name,
    target.subject_line = source.subject_line,
    target.preview_text = source.preview_text,
    target.emails_sent = source.emails_sent,
    target.abuse_reports = source.abuse_reports,
    target.unsubscribed = source.unsubscribed,
    target.send_time = source.send_time,
    target.opens_total = source.opens_total,
    target.unique_opens = source.unique_opens,
    target.open_rate = source.open_rate,
    target.last_open = source.last_open,
    target.clicks_total = source.clicks_total,
    target.unique_clicks = source.unique_clicks,
    target.unique_subscriber_clicks = source.unique_subscriber_clicks,
    target.click_rate = source.click_rate,
    target.last_click = source.last_click,
    target.bounce_rate = source.bounce_rate,
    target.unopen_rate = source.unopen_rate,
    target.unsub_rate = source.unsub_rate,
    target.abuse_rate = source.abuse_rate,
    target.sub_rate = source.sub_rate
WHEN NOT MATCHED THEN
  INSERT (
    id, campaign_title, type, list_id, list_is_active, list_name, subject_line, preview_text, emails_sent, abuse_reports, unsubscribed,
    send_time, opens_total, unique_opens, open_rate, last_open, clicks_total, unique_clicks, unique_subscriber_clicks, click_rate, 
    last_click, bounce_rate, unopen_rate, unsub_rate, abuse_rate, sub_rate
  ) 
  VALUES (
    source.id, source.campaign_title, source.type, source.list_id, source.list_is_active, source.list_name, source.subject_line,
    source.preview_text, source.emails_sent, source.abuse_reports, source.unsubscribed, source.send_time, source.opens_total,
    source.unique_opens, source.open_rate, source.last_open, source.clicks_total, source.unique_clicks, source.unique_subscriber_clicks,
    source.click_rate, source.last_click, source.bounce_rate, source.unopen_rate, source.unsub_rate, source.abuse_rate, source.sub_rate
  );


In [ ]:
%sql
OPTIMIZE common.MailChimp_AFDR_Campaign_ReportData ZORDER BY (id, send_time);

In [ ]:
import json

all_open_details = []

for cid in cids:
    try:
        offset = 0
        limit = 1000
        while True:
            response = client.reports.get_campaign_open_details(cid, count=limit, offset=offset)
            details = response['members']
            if details:
                all_open_details.extend(details)
                offset += limit
            else:
                break
    except Exception as error:
        print("Error: {}".format(error))

open_data = []

for details in all_open_details:
    open_details = {
        "campaign_id": details.get('campaign_id', None),
        "list_id": details.get('list_id', None),
        "list_is_active": details.get('list_is_active', None),
        "contact_status": details.get('contact_status', None),
        "email_id": details.get('email_id', None),
        "email_address": details.get('email_address', None),
        "vip": details.get('vip', None),
        "opens_count": details.get('opens_count', None),
        "opens": details.get('opens', None)
    }

    # Handle merge_fields
    try:
        open_details["FNAME"] = details['merge_fields']['FNAME']
    except KeyError:
        open_details["FNAME"] = None

    try:
        open_details["LNAME"] = details['merge_fields']['LNAME']
    except KeyError:
        open_details["LNAME"] = None

    try:
        open_details["ADDRESS"] = details['merge_fields']['ADDRESS']
    except KeyError:
        open_details["ADDRESS"] = None

    try:
        open_details["PHONE"] = details['merge_fields']['PHONE']
    except KeyError:
        open_details["PHONE"] = None

    try:
        open_details["FIRMNAME"] = details['merge_fields']['FIRMNAME']
    except KeyError:
        open_details["FIRMNAME"] = None

    try:
        open_details["FIRMCLASS"] = details['merge_fields']['FIRMCLASS']
    except KeyError:
        open_details["FIRMCLASS"] = None

    open_data.append(open_details)

# # Print the formatted JSON
# print(json.dumps(open_data, indent=4))


In [ ]:
# Block 2: Create a Pandas DataFrame from Processed Data
df = pd.DataFrame(open_data)

# Explicitly convert the 'ADDRESS' column to strings in the Pandas DataFrame
df['ADDRESS'] = df['ADDRESS'].astype(str)

# Create a Spark DataFrame from the Pandas DataFrame
df = spark.createDataFrame(df)

df.createOrReplaceTempView("temp_table4")

In [ ]:
%sql
MERGE INTO common.MailChimp_AFDR_Campaign_OpenDetails AS target
USING temp_table4 AS source
ON target.campaign_id = source.campaign_id AND target.email_id = source.email_id
WHEN MATCHED THEN
  UPDATE SET 
    target.list_id = source.list_id,
    target.list_is_active = source.list_is_active,
    target.contact_status = source.contact_status,
    target.email_address = source.email_address,
    target.vip = source.vip,
    target.opens_count = source.opens_count,
    target.opens = source.opens,
    target.FNAME = source.FNAME,
    target.LNAME = source.LNAME,
    target.ADDRESS = source.ADDRESS,
    target.PHONE = source.PHONE,
    target.FIRMNAME = source.FIRMNAME,
    target.FIRMCLASS = source.FIRMCLASS
WHEN NOT MATCHED THEN
  INSERT (
    campaign_id, list_id, list_is_active, contact_status, email_id, email_address, vip, opens_count, opens,
    FNAME, LNAME, ADDRESS, PHONE, FIRMNAME, FIRMCLASS
  ) 
  VALUES (
    source.campaign_id, source.list_id, source.list_is_active, source.contact_status, source.email_id,
    source.email_address, source.vip, source.opens_count, source.opens, source.FNAME, source.LNAME,
    source.ADDRESS, source.PHONE, source.FIRMNAME, source.FIRMCLASS
  );


In [ ]:
%sql
OPTIMIZE common.MailChimp_AFDR_Campaign_OpenDetails ZORDER BY (campaign_id, email_id, opens);

In [ ]:
urls_clicked_all = []

for cid in cids:
    try:
        offset = 0
        limit = 1000
        while True:
            response = client.reports.get_campaign_click_details(cid, count=limit, offset=offset)
            clicked = response["urls_clicked"]
            if clicked:
                urls_clicked_all.extend(clicked)
                offset += limit
            else:
                break
    except Exception as error:
        print("Error: {}".format(error))

click_data = []
for urls in urls_clicked_all:
    urls_clicked = {
    'id': urls['id'],
    'url': urls['url'],
    'total_clicks': urls['total_clicks'],
    'click_percentage': urls['click_percentage'],
    'unique_clicks': urls['unique_clicks'],
    'unique_click_percentage': urls['unique_click_percentage'],
    'last_click': urls['last_click'],
    'campaign_id': urls['campaign_id']
    }
    click_data.append(urls_clicked)
# print(json.dumps(click_data, indent=4)) 

In [ ]:
df = pd.DataFrame(click_data)
df = spark.createDataFrame(df)
df.createOrReplaceTempView("temp_table5")

In [ ]:
%sql
MERGE INTO common.MailChimp_AFDR_Campaign_ClickDetails AS target
USING temp_table5 AS source
ON target.id = source.id AND target.campaign_id = source.campaign_id
WHEN MATCHED THEN
  UPDATE SET 
    target.url = source.url,
    target.total_clicks = source.total_clicks,
    target.click_percentage = source.click_percentage,
    target.unique_clicks = source.unique_clicks,
    target.unique_click_percentage = source.unique_click_percentage,
    target.last_click = source.last_click
WHEN NOT MATCHED THEN
  INSERT (
    id, url, total_clicks, click_percentage, unique_clicks, unique_click_percentage, last_click, campaign_id
  ) 
  VALUES (
    source.id, source.url, source.total_clicks, source.click_percentage, source.unique_clicks, source.unique_click_percentage, source.last_click, source.campaign_id
  );


In [ ]:
%sql
OPTIMIZE common.MailChimp_AFDR_Campaign_ClickDetails ZORDER BY (id, campaign_id, url);

In [ ]:
unsub_all = []
for cid in cids:
    try:
        offset = 0
        limit = 1000
        while True:
            response = client.reports.get_unsubscribed_list_for_campaign(cid, offset=offset, count=limit)
            unsubs = response['unsubscribes']
            if unsubs:
                unsub_all.extend(unsubs)
                offset += limit
            else:
                break
    except Exception as error:
        print("Error: {}".format(error))

unsub_data = []
for unsub in unsub_all:
    unsub_details = {'email_id': unsub['email_id'],
    'email_address': unsub['email_address'],
    "FNAME": unsub['merge_fields']['FNAME'],
        "LNAME": unsub['merge_fields']['LNAME'],
        "ADDRESS": unsub['merge_fields']['ADDRESS'],
        "PHONE": unsub['merge_fields']['PHONE'],
        "FIRMNAME": unsub['merge_fields']['FIRMNAME'],
        "FIRMCLASS": unsub['merge_fields']['FIRMCLASS'],
    'vip': unsub['vip'],
    'timestamp': unsub['timestamp'],
    'reason': unsub['reason'],
    'campaign_id': unsub['campaign_id'],
    'list_id': unsub['list_id'],
    'list_is_active': unsub['list_is_active']
    }
    unsub_data.append(unsub_details)
# print(json.dumps(unsub_data, indent=2))

In [ ]:
df = pd.DataFrame(unsub_data)
df = spark.createDataFrame(df)
df.createOrReplaceTempView("temp_table6")

In [ ]:
%sql
MERGE INTO common.MailChimp_AFDR_Campaign_UnsubscribedDetails AS target
USING temp_table6 AS source
ON target.email_id = source.email_id AND target.campaign_id = source.campaign_id
WHEN MATCHED THEN
  UPDATE SET 
    target.email_address = source.email_address,
    target.FNAME = source.FNAME,
    target.LNAME = source.LNAME,
    target.ADDRESS = source.ADDRESS,
    target.PHONE = source.PHONE,
    target.FIRMNAME = source.FIRMNAME,
    target.FIRMCLASS = source.FIRMCLASS,
    target.vip = source.vip,
    target.timestamp = source.timestamp,
    target.reason = source.reason,
    target.list_id = source.list_id,
    target.list_is_active = source.list_is_active
WHEN NOT MATCHED THEN
  INSERT (
    email_id, email_address, FNAME, LNAME, ADDRESS, PHONE, FIRMNAME, FIRMCLASS, vip, timestamp, reason, campaign_id, list_id, list_is_active
  ) 
  VALUES (
    source.email_id, source.email_address, source.FNAME, source.LNAME, source.ADDRESS, source.PHONE, source.FIRMNAME, source.FIRMCLASS,
    source.vip, source.timestamp, source.reason, source.campaign_id, source.list_id, source.list_is_active
  );


In [ ]:
# hash emails
subscriber_hashes = [hashlib.md5(email.lower().encode()).hexdigest() for email in emails]

In [ ]:
all_activity = []
for subscriber_hash in subscriber_hashes:
     try:
        # pagination is not supported with this method
        response = client.lists.get_list_member_activity(list_id, subscriber_hash)
        activity = response['activity']
        email_id = response['email_id']
        
        if activity:
            for entry in activity:
                entry['email_id'] = email_id
            all_activity.extend(activity)
        else:
            break
     except ApiClientError as error:
        print("Error: {}".format(error.text))


In [ ]:
df = pd.DataFrame(all_activity)
clicks = spark.createDataFrame(df)
clicks = clicks.select("action", "timestamp", "campaign_id", "title", "email_id", "url").distinct()
# display(clicks)

In [ ]:
clicks.createOrReplaceTempView("temp_table7")

In [ ]:
%sql
MERGE INTO common.MailChimp_AFDR_Campaign_MemberActivity AS target
USING temp_table7 AS source 

ON target.email_id = source.email_id 
   AND target.timestamp = source.timestamp
   AND target.campaign_id = source.campaign_id
   AND target.url = source.url

WHEN MATCHED THEN
    UPDATE SET 
        target.action = source.action,
        target.title = source.title

WHEN NOT MATCHED THEN
    INSERT (
        action,  
        timestamp,  
        campaign_id, 
        title, 
        email_id, 
        url
    )
    VALUES (
        source.action, 
        source.timestamp, 
        source.campaign_id, 
        source.title, 
        source.email_id, 
        source.url
    );


In [ ]:
%sql
OPTIMIZE common.MailChimp_AFDR_Campaign_MemberActivity ZORDER BY (action, url);

In [ ]:
df = spark.table('common.MailChimp_AFDR_Members')
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/aletia@bondcliq.com/MC_Import.csv")

from pyspark.sql import functions as F

# Rename the columns in df1 before joining
df1_renamed = df1.withColumnRenamed("email", "df1_email").withColumnRenamed("firmclass", "df1_firmclass")

# Join df with df1_renamed on their respective email columns
joined_df = df.join(df1_renamed.select("df1_email", "df1_firmclass"), df["email_address"] == df1_renamed["df1_email"], how="left")

# Update the FIRMCLASS column with values from df1's firmclass
updated_df = joined_df.withColumn("FIRMCLASS", F.coalesce(joined_df["df1_firmclass"], joined_df["FIRMCLASS"]))

# Add an updated_flag column: 1 if FIRMCLASS was updated, 0 otherwise
updated_df = updated_df.withColumn("updated_flag", F.when(joined_df["df1_firmclass"].isNotNull(), 1).otherwise(0))

# Drop df1's firmclass and email columns if they're not required anymore
final_df = updated_df.drop("df1_firmclass").drop("df1_email")

table_path = "temp_db.mc_ClassUpdate_members_AFDR"
final_df.write.format("delta").mode("overwrite").saveAsTable(table_path)